<table style="width:100%">
  <tr>
    <td style="background-color:#EBF5FB; border: 1px solid #CFCFCF">
      <b>National generation capacity: Check notebook</b>
      <ul>
          <li><a href="main.ipynb">Main notebook</a></li>
          <li><a href="processing.ipynb">Processing notebook</a></li>
          <li>Check notebook (this)</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/national_generation_capacity">National Generation Capacity Datapackage</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory notes](#1.-Introductory-notes)
* [2. Script setup](#2.-Script-setup)
* [3. Import of processed data](#3.-Import-of-processed-data)
* [4. Visualisation of results for different energy source levels](#4.-Visualisation-of-results-for-different-energy-source-levels)
	* [4.1 Energy source level 1](#4.1-Energy-source-level-1)
		* [4.1.1 Table](#4.1.1-Table)
		* [4.1.2 Bokeh chart](#4.1.2-Bokeh-chart)
	* [4.2 Energy source level 2](#4.2-Energy-source-level-2)
		* [4.2.1 Table](#4.2.1-Table)
		* [4.2.2 Bokeh chart](#4.2.2-Bokeh-chart)
	* [4.3 Energy source level 3](#4.3-Energy-source-level-3)
		* [4.3.1 Table](#4.3.1-Table)
		* [4.3.2 Bokeh chart](#4.3.2-Bokeh-chart)
	* [4.4 Technology level](#4.4-Technology-level)
		* [4.4.1 Table](#4.4.1-Table)
		* [4.4.2 Bokeh chart](#4.4.2-Bokeh-chart)
* [5. Comparison of total capacity for energy source levels](#5.-Comparison-of-total-capacity-for-energy-source-levels)
	* [5.1 Calculation of total capacity for energy source levels](#5.1-Calculation-of-total-capacity-for-energy-source-levels)
	* [5.2 Identifcation of capacity differences for energy source levels](#5.2-Identifcation-of-capacity-differences-for-energy-source-levels)


# 1. Introductory notes

The notebook extends the [processing notebook](processing.ipynb) to make visualisations and perform consistency checks.

# 2. Script setup

In [ ]:
import pandas as pd
import numpy as np
import os.path
import logging

from bokeh.charts import Bar, output_file, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter
from bokeh.charts.attributes import color

output_notebook()

%matplotlib inline

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%d %b %Y %H:%M:%S'
    )

logger = logging.getLogger()

In [1]:
import pandas as pd
import numpy as np
import os.path
import bokeh.plotting as plt
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

Loading BokehJS ...

# 3. Import of processed data

The processed data is imported at this stage. This requires that the [processing notebook](processing.ipynb) is compiled prior to this step. Otherwise, no or an old dataset is imported.

In [92]:
data_file = 'national_generation_capacity_stacked.csv'
filepath = os.path.join('output', data_file)
data = pd.read_csv(filepath, index_col=0)

data.head()

,technology,source,source_type,weblink,year,type,country,capacity_definition,capacity,comment,energy_source_level_0,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology_level
ID,,,,,,,,,,,,,,,
0,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2014,Installed capacity in MW,AT,Gross capacity,7243.634,NaN,False,True,False,False,False
1,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2015,Installed capacity in MW,AT,Gross capacity,7059.065,NaN,False,True,False,False,False
2,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2016,Installed capacity in MW,AT,Gross capacity,7323.000,NaN,False,True,False,False,False
3,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2017,Installed capacity in MW,AT,Gross capacity,6469.000,NaN,False,True,False,False,False
4,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2018,Installed capacity in MW,AT,Gross capacity,6492.000,NaN,False,True,False,False,False


# 4. Visualisation of results for different energy source levels

In the following, national generation capacity is compared for the different energy source levels. Due to the size of the dataset regarding countries and compiled datasources, the following tables and figures tend to be rather confusing. Therefore, we provide the following option to limit the visualisation of the results to a selection of countries and years. If the corresponding subset is empty, all values will be considered. 

In [ ]:
country_subset = ['DE', 'FR', 'BE', 'NL', 'IT']
year_subset = [2013, 2014, 2015, 2016]

In [ ]:
data_selection = pd.DataFrame()

if len(country_subset) == 0:
    data_selection = data
else:
    for country in country_subset:
        for year in year_subset:
            if len(data_selection) == 0:
                data_selection = data[(data.country == country) &
                                      (data.year == year)]
            else:
                data_selection = data_selection.append(data[(data.country == country) &
                                                            (data.year == year)])

data_selection

To improve the data visualisation in Bokeh the colors of the defined technologies is specified explicitly. The user is free to adjust or refine the color definition using the following parameter. The color names are defined [here](http://www.w3schools.com/colors/colors_names.asp).

In [ ]:
colormap = {
    'Fossil fuels': 'Black',
    'Lignite': 'SaddleBrown',
    'Hard coal': 'Black',
    'Oil': 'Violet',
    'Natural gas': 'IndianRed',
    'Combined cycle': '#d57676',
    'Gas turbine': '#e19d9d',
    'Other and unknown natural gas': '#c33c3c',
    'Differently categorized natural gas': 'IndianRed',
    'Non-renewable waste': 'SandyBrown',
    'Mixed fossil fuels': 'LightGray',
    'Other fossil fuels': 'DarkGray',
    'Differently categorized fossil fuels': 'Gray',
    'Nuclear': 'Red',
    'Renewable energy sources': 'Green',
    'Hydro': 'Navy',
    'Run-of-river': '#0000b3',
    'Reservoir': '#0000e6',
    'Reservoir including pumped storage': '#0000e6',
    'Pumped storage': '#1a1aff',
    'Pumped storage with natural inflow': '#1a1aff',
    'Differently categorized hydro': 'Navy',
    'Wind': 'SkyBlue',
    'Onshore': 'LightSkyBlue',
    'Offshore': 'DeepSkyBlue',
    'Differently categorized wind': 'SkyBlue',
    'Solar': 'Yellow',
    'Photovoltaics': '#ffff33',
    'Concentrated solar power': '#ffff66',
    'Differently categorized solar': 'Yellow',
    'Geothermal': 'DarkRed',
    'Marine': 'Blue',
    'Bioenergy and renewable waste': 'Green',
    'Biomass and biogas': '#00b300',
    'Sewage and landfill gas': '#00e600',
    'Other bioenergy and renewable waste': 'Green',
    'Differently categorized renewable energy sources': 'Green',
    'Other or unspecified energy sources': 'Orange',
}

# New stuff

In [174]:
input_co = 'DE'
input_year = [2015,2016,2017]

In [175]:
co_subset = data['country'] == input_co
y_subset = data['year'].isin(input_year)
energy_source_level = data['energy_source_level_3'] == True
ret_data = data.loc[co_subset & y_subset & energy_source_level]

In [176]:
sources = list(ret_data.source.unique())
technologies = list(ret_data.technology.unique())
years = list(ret_data.year.unique().astype(str))

In [177]:
source_data = pd.pivot_table(ret_data, 
                          index=('year','source'),
                          columns='technology',
                          values='capacity',
                          aggfunc=sum,
                          margins=False)

source_data.head()

technology                          Biomass and biogas  \
year source                                              
2015 ENTSO-E Data Portal                       6915.00   
     ENTSO-E Power Statistics                  6804.36   
     ENTSO-E SOAF                              6532.00   
     ENTSO-E Transparency Platform             6787.00   
     EUROSTAT                                  5875.00   

technology                          Differently categorized fossil fuels  \
year source                                                                
2015 ENTSO-E Data Portal                                         78746.0   
     ENTSO-E Power Statistics                                        0.0   
     ENTSO-E SOAF                                                    0.0   
     ENTSO-E Transparency Platform                                   0.0   
     EUROSTAT                                                    86623.0   

technology                          Differently categorized renewable energy sources  \
year source                                                                            
2015 ENTSO-E Data Portal                                                      559.00   
     ENTSO-E Power Statistics                                                 511.69   
     ENTSO-E SOAF                                                             590.00   
     ENTSO-E Transparency Platform                                            514.00   
     EUROSTAT                                                                    NaN   

technology                          Geothermal  Hard coal     Hydro  Lignite  \
year source                                                                    
2015 ENTSO-E Data Portal                   NaN        NaN   9610.00      NaN   
     ENTSO-E Power Statistics            33.44    26479.6  10365.75  20929.0   
     ENTSO-E SOAF                          NaN    27000.0  10230.00  20800.0   
     ENTSO-E Transparency Platform       33.00    26190.0  14149.15  21160.0   
     EUROSTAT                            26.00        NaN  11399.00      NaN   

technology                          Marine  Mixed fossil fuels  Natural gas  \
year source                                                                   
2015 ENTSO-E Data Portal               NaN                 NaN          NaN   
     ENTSO-E Power Statistics          0.0              3149.5      28325.8   
     ENTSO-E SOAF                      NaN                 0.0      26700.0   
     ENTSO-E Transparency Platform     0.0                 0.0      31496.0   
     EUROSTAT                          0.0                 NaN          NaN   

technology                          Non-renewable waste  Nuclear     Oil  \
year source                                                                
2015 ENTSO-E Data Portal                            NaN  10793.0     NaN   
     ENTSO-E Power Statistics                    810.75  10793.0  2140.3   
     ENTSO-E SOAF                                   NaN  10800.0  3900.0   
     ENTSO-E Transparency Platform                 0.00  12068.0  4511.0   
     EUROSTAT                                   2877.00  10799.0     NaN   

technology                          Other bioenergy and renewable waste  \
year source                                                               
2015 ENTSO-E Data Portal                                            NaN   
     ENTSO-E Power Statistics                                    810.75   
     ENTSO-E SOAF                                                   NaN   
     ENTSO-E Transparency Platform                              1685.00   
     EUROSTAT                                                   1592.00   

technology                          Other fossil fuels  \
year source                                              
2015 ENTSO-E Data Portal                           NaN   
     ENTSO-E Power Statistics                      0.0   
     ENTSO-E SOAF                    

In [179]:
x_axis = list(source_data.index)
source_data.reset_index(inplace=True)
source_data["year"] = source_data["year"].astype(str)
source = ColumnDataSource(source_data)

In [181]:
x_axis

[(2015, 'ENTSO-E Data Portal'),
 (2015, 'ENTSO-E Power Statistics'),
 (2015, 'ENTSO-E SOAF'),
 (2015, 'ENTSO-E Transparency Platform'),
 (2015, 'EUROSTAT'),
 (2016, 'ENTSO-E Power Statistics'),
 (2016, 'ENTSO-E Transparency Platform'),
 (2016, 'EUROSTAT'),
 (2017, 'ENTSO-E Power Statistics'),
 (2017, 'ENTSO-E Transparency Platform')]

In [182]:
p = plt.figure(x_range=FactorRange(*x_axis), title=input_co, tools="")

p.vbar_stack(technologies, x='x', width=0.9, source=source, legend_label=technologies)

plt.show(p)

ValueError: expected an element of either Seq(String), Seq(Tuple(String, String)) or Seq(Tuple(String, String, String)), got [(2015, 'ENTSO-E Data Portal'), (2015, 'ENTSO-E Power Statistics'), (2015, 'ENTSO-E SOAF'), (2015, 'ENTSO-E Transparency Platform'), (2015, 'EUROSTAT'), (2016, 'ENTSO-E Power Statistics'), (2016, 'ENTSO-E Transparency Platform'), (2016, 'EUROSTAT'), (2017, 'ENTSO-E Power Statistics'), (2017, 'ENTSO-E Transparency Platform')]

## 4.1 Energy source level 1

### 4.1.1 Table

In [ ]:
pivot_capacity_level1 = pd.pivot_table(data_selection[data_selection.energy_source_level_1 == True],
                                       index=('country','year','source'),
                                       columns='technology',
                                       values='capacity',
                                       aggfunc=sum,
                                       margins=False)

pivot_capacity_level1

### 4.1.2 Bokeh chart

Please use the zoom and hover option to inspect the data graphically.

In [ ]:
data_energy_level_1 = data_selection[data_selection.energy_source_level_1 == True].copy()
data_energy_level_1['color'] = 'White'
data_energy_level_1['color'] = data_energy_level_1['technology'].map(colormap)

bar = Bar(data_energy_level_1, values='capacity', label=['country', 'year', 'source'], stack='technology',
          title="National capacity by type of energy source", tools="pan,wheel_zoom,box_zoom,reset,hover,save",
          legend='top_right', 
          plot_width=1600, plot_height=800,
#          color=color(columns='technology', palette=['Black', 'Red', 'Green', 'Orange'], sort=False))
          color='color')
bar._yaxis.formatter = NumeralTickFormatter(format="00,000 MW")

hover = bar.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Country", "@country"),
                  ("Year", "@year"),
                  ("Source", "@source"),
                  ("Category", "@technology"),
                  ("Capacity", "@height{00,000.00} MW"),
]

show(bar)

In [ ]:
test_df = pd.DataFrame({'country': ["Germany","DK","FR", "Germany","DK","FR"],
                        'capacity': np.array([5,2,4, 5, 3, 7]),
                        'technology': ["a","a","a", "b", "b", "b"]
                       })

#test_df["index"] = range(1, len(test_df)+1)
#new_ticks = dict(zip(test_df["index"], test_df["country"]))
asdf = pd.pivot_table(test_df, index='country', columns='technology', values='capacity')
source = ColumnDataSource(asdf)
asdf.head()

In [ ]:
countries = test_df['country'].unique()
technologies = test_df['technology'].unique()
colors = ["#c9d9d3", "#718dbf"]
p = plt.figure(x_range=countries)
p.vbar_stack(technologies, x='country', width=0.9, source=source, color=colors)
plt.show(p)

In [ ]:
p = plt.figure(title="simple line example", x_axis_label='country', y_axis_label='capacity',
              plot_width=950,  x_range=test_df["country"])
p.vbar(test_df['technology'], x=test_df["country"], width=0.5)
#p.xaxis.ticker = test_df["index"]
#p.xaxis.major_label_overrides = new_ticks

plt.show(p)

In [ ]:
df = data_energy_level_1.loc[data_energy_level_1["source"] == "ENTSO-E SOAF", :]
df = df[df['country'] == 'DE']
df.head()

In [ ]:
df["index"] = range(1, len(df)+1)
new_ticks = dict(zip(df["index"], df["year"]))
source = ColumnDataSource(df)
df.head()

In [ ]:
p = plt.figure(title="simple line example", x_axis_label='country', y_axis_label='capacity',
              plot_width=950)
p.vbar_stack(x='index', stackers='capacity', width=0.5, source=source)
#p.xaxis.ticker = test_df["index"]
#p.xaxis.major_label_overrides = new_ticks

plt.show(p)

In [3]:
test_df = pd.DataFrame({'country': ["Germany","DK","FR", "Germany","DK","FR"],
                        'capacity': np.array([5,2,4, 5, 3, 7]),
                        'technology': ["a","a","a", "b", "b", "b"]
                       })

#test_df["index"] = range(1, len(test_df)+1)
#new_ticks = dict(zip(test_df["index"], test_df["country"]))
asdf = pd.pivot_table(test_df, index='country', columns='technology', values='capacity')
source = ColumnDataSource(asdf)
asdf.head()

technology,a,b
country,,
DK,2,3
FR,4,7
Germany,5,5


In [36]:
countries = list(asdf.index)
technologies = list(asdf.columns)

In [38]:
colors = ["#c9d9d3", "#718dbf"]

data = {'countries' : countries,
        'a'   : [2, 1, 4],
        'b'   : [5, 3, 4]}

p = plt.figure(x_range=countries, plot_height=250, title="bla",
           toolbar_location=None, tools="")

p.vbar_stack(technologies, x='countries', width=0.9, color=colors, source=data,
            legend_label=technologies)

plt.show(p)

In [80]:
countries = list(asdf.index)
technologies = list(asdf.columns)

In [81]:
years = ['2015','2016']
x_axis = [ (c, y) for c in countries for y in years ]
x_axis

[('DK', '2015'),
 ('DK', '2016'),
 ('FR', '2015'),
 ('FR', '2016'),
 ('Germany', '2015'),
 ('Germany', '2016')]

In [82]:
y1 = [i for i in range(1,len(x_axis)+1)]
y2 = [1+ i/2 for i in range(1,len(x_axis)+1)]

In [83]:
data = dict(x=x_axis, y1=y1, y2=y2)
source = ColumnDataSource(data)
data

{'x': [('DK', '2015'),
  ('DK', '2016'),
  ('FR', '2015'),
  ('FR', '2016'),
  ('Germany', '2015'),
  ('Germany', '2016')],
 'y1': [1, 2, 3, 4, 5, 6],
 'y2': [1.5, 2.0, 2.5, 3.0, 3.5, 4.0]}

In [86]:
tech = ['y1','y2']
colors = ["#c9d9d3", "#718dbf"]

In [89]:
p = plt.figure(x_range=FactorRange(*x_axis), title="bla",tools="")

#p.vbar(x='x', top='y1', width=0.9, source=source)
p.vbar_stack(tech, x='x', width=0.9, source=source, color = colors,
            legend_label=tech)

plt.show(p)

## 4.2 Energy source level 2

### 4.2.1 Table

In [ ]:
pivot_capacity_level2 = pd.pivot_table(data_selection[data_selection.energy_source_level_2 == True],
                                       index=('country','year','source'),
                                       columns='technology',
                                       values='capacity',
                                       aggfunc=sum,
                                       margins=False)

pivot_capacity_level2

### 4.2.2 Bokeh chart

Please use the zoom and hover option to inspect the data graphically.

In [ ]:
data_energy_level_2 = data_selection[data_selection.energy_source_level_2 == True].copy()
data_energy_level_2['color'] = 'White'
data_energy_level_2['color'] = data_energy_level_2['technology'].map(colormap)

bar = Bar(data_energy_level_2, values='capacity', label=['country', 'year', 'source'], stack='technology',
          title="National capacity by energy source", tools="pan,wheel_zoom,box_zoom,reset,hover,save",
          legend='top_right', 
          plot_width=1600, plot_height=800,
          color='color'
         )
bar._yaxis.formatter = NumeralTickFormatter(format="00,000 MW")

hover = bar.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Country", "@country"),
                  ("Year", "@year"),
                  ("Source", "@source"),
                  ("Category", "@technology"),
                  ("Capacity", "@height{00,000.00} MW"),
]

show(bar)

## 4.3 Energy source level 3

### 4.3.1 Table

In [ ]:
pivot_capacity_level3 = pd.pivot_table(data_selection[data_selection.energy_source_level_3 == True],
                                       index=('country', 'year', 'source'),
                                       columns='technology',
                                       values='capacity',
                                       aggfunc=sum,
                                       margins=False)

pivot_capacity_level3

### 4.3.2 Bokeh chart

Please use the zoom and hover option to inspect the data graphically.

In [ ]:
data_energy_level_3 = data_selection[data_selection.energy_source_level_3 == True].copy()
data_energy_level_3['color'] = 'White'
data_energy_level_3['color'] = data_energy_level_3['technology'].map(colormap)

bar = Bar(data_energy_level_3, values='capacity', label=['country', 'year', 'source'], stack='technology',
          title="National capacity by energy source", tools="pan,wheel_zoom,box_zoom,reset,hover,save",
#          legend='top_right', 
          plot_width=1600, plot_height=800,
          color='color'
         )
bar._yaxis.formatter = NumeralTickFormatter(format="00,000 MW")

hover = bar.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Country", "@country"),
                  ("Year", "@year"),
                  ("Source", "@source"),
                  ("Category", "@technology"),
                  ("Capacity", "@height{00,000.00} MW"),
]

show(bar)

## 4.4 Technology level

### 4.4.1 Table

In [ ]:
pivot_capacity_techlevel = pd.pivot_table(data_selection[data_selection.technology_level == True],
                                          index=('country', 'year', 'source'),
                                          columns='technology',
                                          values='capacity',
                                          aggfunc=sum,
                                          margins=False)

pivot_capacity_techlevel

### 4.4.2 Bokeh chart

Please use the zoom and hover option to inspect the data graphically.

In [ ]:
data_technology_level = data_selection[data_selection.technology_level == True].copy()
data_technology_level['color'] = 'White'
data_technology_level['color'] = data_technology_level['technology'].map(colormap)

bar = Bar(data_technology_level, values='capacity', label=['country', 'year', 'source'], stack='technology',
          title="National capacity by energy source and technology", tools="pan,wheel_zoom,box_zoom,reset,hover,save",
#          legend='top_right', 
          plot_width=1600, plot_height=800,
          color='color'
         )
bar._yaxis.formatter = NumeralTickFormatter(format="00,000 MW")

hover = bar.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Country", "@country"),
                  ("Year", "@year"),
                  ("Source", "@source"),
                  ("Category", "@technology"),
                  ("Capacity", "@height{00,000.00} MW"),
]

show(bar)

# 5. Comparison of total capacity for energy source levels

In the following, the installed capacities at the different technology levels are compared to each other. In any case, the total sum of all technologies within a certain technology level should match with other energy source levels. Otherwise the classification of categories to the levels is flawed or the specific data entries are wrong.

Again, the comparison can be done for specific countries, or, if the selection is empty, for all countries.

In [ ]:
country_subset = []
#country_subset = ['DE', 'FR', 'IT', 'ES']

In [ ]:
data_selection = pd.DataFrame()

if len(country_subset) == 0:
    data_selection = data
else:
    for country in country_subset:
        if len(data_selection) == 0:
            data_selection = data[data.country == country]
        else:
            data_selection = data_selection.append(data[data.country == country])

#data_selection

## 5.1 Calculation of total capacity for energy source levels

In [ ]:
# Define the columns for grouping
groupby_selection = ['capacity_definition', 'source', 'year', 'type', 'country']

# Calculate the total capacity of all categories within a certain technology level
capacity_total_0 = pd.DataFrame(data_selection[data_selection['energy_source_level_0'] == True]
                                .groupby(groupby_selection)['capacity'].sum())
capacity_total_1 = pd.DataFrame(data_selection[data_selection['energy_source_level_1'] == True]
                                .groupby(groupby_selection)['capacity'].sum())
capacity_total_2 = pd.DataFrame(data_selection[data_selection['energy_source_level_2'] == True]
                                .groupby(groupby_selection)['capacity'].sum())
capacity_total_3 = pd.DataFrame(data_selection[data_selection['energy_source_level_3'] == True]
                                .groupby(groupby_selection)['capacity'].sum())
capacity_total_tech = pd.DataFrame(data_selection[data_selection['technology_level'] == True]
                                   .groupby(groupby_selection)['capacity'].sum())

# Merge calculated capacity for different technology levels
capacity_total_comparison = pd.DataFrame(capacity_total_0)
capacity_total_comparison = pd.merge(capacity_total_0, capacity_total_1, 
                                     left_index=True, right_index=True, how='left')
capacity_total_comparison = capacity_total_comparison.rename(columns={'capacity_x': 'energy source level 0',
                                                                      'capacity_y': 'energy source level 1'})

capacity_total_comparison = pd.merge(capacity_total_comparison, capacity_total_2, 
                                     left_index=True, right_index=True, how='left')
capacity_total_comparison = pd.merge(capacity_total_comparison, capacity_total_3, 
                                     left_index=True, right_index=True, how='left')
capacity_total_comparison = capacity_total_comparison.rename(columns={'capacity_x': 'energy source level 2',
                                                                      'capacity_y': 'energy source level 3'})

capacity_total_comparison = pd.merge(capacity_total_comparison, capacity_total_tech, 
                                     left_index=True, right_index=True, how='left')
capacity_total_comparison = capacity_total_comparison.rename(columns={'capacity': 'technology level'})

# Define sorting preferences
capacity_total_comparison = capacity_total_comparison.sortlevel(['country', 'year'])

capacity_total_comparison

## 5.2 Identifcation of capacity differences for energy source levels

Identification of differences between energy source levels for each country, source, and year. The difference is relative to the previous energy source level. Generally, differences between the energy source levels should be zero, but could differ in particular for ENTSO-E data.

In [ ]:
capacity_total_difference = capacity_total_comparison.diff(periods=1, axis=1)
capacity_total_difference = capacity_total_difference[(capacity_total_difference['energy source level 1'] > 0.01) | 
                                                      (capacity_total_difference['energy source level 1'] < -0.01) |
                                                     (capacity_total_difference['energy source level 2'] > 0.01) | 
                                                      (capacity_total_difference['energy source level 2'] < -0.01) |
                                                     (capacity_total_difference['energy source level 3'] > 0.01) | 
                                                      (capacity_total_difference['energy source level 3'] < -0.01)|
                                                     (capacity_total_difference['technology level'] > 0.01) | 
                                                      (capacity_total_difference['technology level'] < -0.01)]
capacity_total_difference